In [2]:
import pandas as pd
import numpy as np
import re

In [3]:

df_original = pd.read_csv('df_tidy_simp.csv')
df = df_original.copy()

df.head(5)

,Fundo,%PL,Norm.,Ativo,Garantia,Nota
0,VGIR11,0.053395,NaN,22L1013767,AF SPE,2.0
1,VGIR11,0.051467,NaN,22E1211649,AF Terreno e SPE + CF,3.0
2,VGIR11,0.049767,NaN,23K1511855,AF SPE + Fiança,2.0
3,VGIR11,0.046417,NaN,24A1828538,AF Imóvel + Aval,3.0
4,VGIR11,0.042653,NaN,22K1684666,AF SPE,2.0


In [4]:
# df com 
df = df[['Fundo' , 'Ativo', 'Garantia']]

In [5]:
regex = r'\s*(?:\+|-|,|;|\bou\b|\be\b|\bem\b|\bde\b|\bda\b|\bdos\b|\bdo\b|\be/?ou\b|\(\w+\)|•)\s*'
df['Garantia'] = df['Garantia'].str.replace(r'^\s*(?:-+|•+|GARANTIAS)\s*', '', regex=True)


In [6]:
def limpar_celula(x):
    if not isinstance(x, str):
        return x
    x = x.strip(" ;.•")                    # remove pontuação
    x = re.sub(r'^\d+\s*', '', x)          # remove número no início
    x = re.sub(r'\s*\d+$', '', x)          # remove número no fim
    return x.strip()

In [7]:
df_split = df['Garantia'].str.split(regex, expand=True)

# Aplica limpeza por célula (pontuação + números no início/fim)
df_split = df_split.applymap(limpar_celula)

# Nomeia as colunas
df_split.columns = [f'Garantia_{i+1}' for i in range(df_split.shape[1])]

# Junta com as colunas originais
df_split = pd.concat([df[['Fundo', 'Ativo']], df_split], axis=1)
df_split.iloc[1:10].dropna(axis=1, how='all')

/var/folders/st/r72pjbts3bb1zfh2s1r4pzw00000gn/T/ipykernel_3594/4078454562.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_split = df_split.applymap(limpar_celula)


,Fundo,Ativo,Garantia_1,Garantia_2,Garantia_3,Garantia_4
1,VGIR11,22E1211649,AF Terreno,SPE,CF,None
2,VGIR11,23K1511855,AF SPE,Fiança,None,None
3,VGIR11,24A1828538,AF Imóvel,Aval,None,None
4,VGIR11,22K1684666,AF SPE,None,None,None
5,VGIR11,22D0836679,clean,None,None,None
6,VGIR11,24G1972260,AF Imóvel,CF,Coobrigação,Aval
7,VGIR11,24E1753141,AF Imóvel,Terreno,CF,Aval
8,VGIR11,23J1928151,AF Imóvel,SPE,CF,Fiança
9,VGIR11,21L0002653,AF Terreno,None,None,None


In [8]:
df_split.to_excel('df_split.xlsx', index=False)
df_split.to_csv('df_split.csv', index=False)